In [1]:
import os
import zipfile
import time
import pickle
import gc

import pandas as pd
import numpy as np
from tqdm import tqdm

from _2_1_gen_user_click_features_day import add_user_feature_click_day,add_user_feature_click_hour
from _2_2_gen_feature_click_day_mean import add_feature_click_stats
from _2_3_gen_user_click_rank import add_user_click_rank_day, add_user_click_rank_global, add_user_feature_click_rank_day, add_user_feature_click_rank_global
from _2_4_gen_predict_category_property import add_item_property_lda, add_category_predict_rank, add_property_sim
from _2_5_ctr_smooth import add_29_smooth_ctr,add_29_hour_ctr,add_features_smooth_ctr,add_features_hour_ctr,add_features_cross_history_ctr,add_features_cross_smooth_ctr
from _2_6_gen_CountVector import add_feature_user_property
from _2_7_gen_click_num import add_feature_click_day, add_feature_click_hour,add_feature_click_day_hour, add_cross_feature_click_day
from _2_8_gen_user_pre_click import add_user_feature_pre_click
#from _2_9_gen_feature_interaction import add_feature_interaction_2_order
from _2_10_future import add_user_feature_future

from utils import load_pickle, dump_pickle, get_feature_value, feature_spearmanr, feature_target_spearmanr, addCrossFeature, calibration
from utils import raw_data_path, feature_data_path, cache_pkl_path, analyse

In [ ]:
def gen_all_data_all_features():

    
    all_data = load_pickle(raw_data_path + 'all_data_4567.pkl')
    
    all_data.drop(['item_category_list', 'item_property_list', 'predict_category_property'], axis=1, inplace=True)
    all_data_path = feature_data_path + 'all_data_all_features.pkl'

#1     #         =======
    print("========= 1 ==========")
    all_data = add_user_feature_click_day(all_data)
    all_data = add_user_feature_click_hour(all_data)
    
#2     #         =======
    print("========= 2 ==========")
    all_data = add_feature_click_stats(all_data)
    
#3     #         ======= 
    print("========= 3 ==========")
    all_data = add_user_click_rank_day(all_data)
    all_data = add_user_click_rank_global(all_data)
    all_data = add_user_feature_click_rank_day(all_data)
    all_data = add_user_feature_click_rank_global(all_data)
    

#4     #         ======= 
    print("========= 4 ==========")
    all_data = add_item_property_lda(all_data)
    all_data = add_property_sim(all_data)    
    all_data = add_category_predict_rank(all_data)
    
#5     #         =======
    print("========= 5 ==========")
    all_data = add_29_smooth_ctr(all_data)
    all_data = add_29_hour_ctr(all_data)
    all_data = add_features_smooth_ctr(all_data)
    all_data = add_features_hour_ctr(all_data)
    all_data = add_features_cross_history_ctr(all_data)
    all_data = add_features_cross_smooth_ctr(all_data)
    
#6     #         =======
    print("========= 6 ==========")
    all_data = add_feature_user_property(all_data)
    
#7     #         =======    
    print("========= 7 ==========")
    all_data = add_feature_click_day(all_data)
#    all_data = add_feature_click_hour(all_data)
    all_data = add_feature_click_day_hour(all_data)
    
#8    #          ========
    print("========= 8 ==========")
    all_data = add_user_feature_pre_click(all_data)
    
    
#     10
    print("========= 10 ==========")
    all_data = add_user_feature_future(all_data)

    
    float_cols = [c for c in all_data if all_data[c].dtype == "float64"]
    all_data[float_cols] = all_data[float_cols].astype(np.float32)
    
    str_cols = [c for c in all_data if all_data[c].dtype == "object"]
    all_data[str_cols] = all_data[str_cols].astype(np.int)
    
    print("========= 存储 ==========")
    dump_pickle(all_data, all_data_path,protocol=4)

    return all_data

In [ ]:
if __name__ == '__main__':
    all_data = gen_all_data_all_features()
    pd.set_option('display.max_rows', None)
    print(all_data.columns)

  0%|          | 0/9 [00:00<?, ?it/s]

========= 1 ==========


/home/qwc/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
  0%|          | 0/7 [00:00<?, ?it/s]

========= 2 ==========


100%|██████████| 7/7 [00:38<00:00,  5.54s/it]


...

  0%|          | 0/8 [00:00<?, ?it/s]

========= 6 ==========


  0%|          | 0/8 [00:00<?, ?it/s]

========= 7 ==========


 62%|██████▎   | 5/8 [01:54<01:08, 22.90s/it]

# 添加2-6不分特征

In [3]:
from _2_6_gen_CountVector import add_feature_user_property_hour
all_data_path = feature_data_path + 'all_data_all_features.pkl'
all_data = load_pickle(all_data_path)
all_data = add_feature_user_property_hour(all_data)

float_cols = [c for c in all_data if all_data[c].dtype == "float64"]
all_data[float_cols] = all_data[float_cols].astype(np.float32)
str_cols = [c for c in all_data if all_data[c].dtype == "object"]
all_data[str_cols] = all_data[str_cols].astype(np.int)

dump_pickle(all_data, feature_data_path + 'all_data_all_features_new.pkl',protocol=4)

  0%|          | 0/7 [00:00<?, ?it/s]/home/qwc/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 7/7 [10:56<00:00, 93.81s/it] 


# 添加2-10部分特征

In [ ]:
from _2_10_future import add_user_feature_before
all_data_path = feature_data_path + 'all_data_all_features.pkl'
all_data = load_pickle(all_data_path)
all_data = add_user_feature_before(all_data)

float_cols = [c for c in all_data if all_data[c].dtype == "float64"]
all_data[float_cols] = all_data[float_cols].astype(np.float32)
str_cols = [c for c in all_data if all_data[c].dtype == "object"]
all_data[str_cols] = all_data[str_cols].astype(np.int)

dump_pickle(all_data, feature_data_path + 'all_data_all_features_new.pkl',protocol=4)

## 添加2-7交叉特征每日点击 
## 添加2-9特征与day交叉

In [2]:
from _2_7_gen_click_num import add_cross_feature_click_day
from _2_9_gen_feature_interaction import add_feature_interaction_day_1_order, add_feature_interaction_day_2_order

all_data_path = feature_data_path + 'all_data_all_features_new.pkl'
all_data = load_pickle(all_data_path)
all_data = add_cross_feature_click_day(all_data)
all_data = add_feature_interaction_day_1_order(all_data)
all_data = add_feature_interaction_day_2_order(all_data)

dump_pickle(all_data, feature_data_path + 'all_data_all_features_new_0512.pkl',protocol=4)

NameError: name 'data' is not defined

In [4]:
pd.set_option('display.max_rows', None)
all_data.isnull().sum()

index                                                               0
instance_id                                                         0
item_id                                                             0
item_brand_id                                                       0
item_city_id                                                        0
item_price_level                                                    0
item_sales_level                                                    0
item_collected_level                                                0
item_pv_level                                                       0
user_id                                                             0
user_gender_id                                                      0
user_age_level                                                      0
user_occupation_id                                                  0
user_star_level                                                     0
context_id          

In [9]:
data.isnull().sum()

index                                                               0
instance_id                                                         0
item_id                                                             0
item_brand_id                                                       0
item_city_id                                                        0
item_price_level                                                    0
item_sales_level                                                    0
item_collected_level                                                0
item_pv_level                                                       0
user_id                                                             0
user_gender_id                                                      0
user_age_level                                                      0
user_occupation_id                                                  0
user_star_level                                                     0
context_id          

In [10]:
all_data.dtypes

index                                                            int64
instance_id                                                      int64
item_id                                                          int64
item_brand_id                                                    int64
item_city_id                                                     int64
item_price_level                                                 int64
item_sales_level                                                 int64
item_collected_level                                             int64
item_pv_level                                                    int64
user_id                                                          int64
user_gender_id                                                   int64
user_age_level                                                   int64
user_occupation_id                                               int64
user_star_level                                                  int64
contex

In [12]:
all_data[all_data.user_id == 50153023443529][['context_timestamp','item_brand_id' ,'user_' + 'category2_label' + '_future_15min']]

,context_timestamp,item_brand_id,user_category2_label_future_15min
190276,1536240332,5510998113992316938,2
265603,1536239858,8764113826665518348,3
475314,1536241480,7147251880558104750,0
543278,1536239177,7147251880558104750,3
683293,1536239046,8764113826665518348,4
832258,1536240799,6030170612722734811,1
1010442,1536239046,1326455302495217018,4
1369495,1536240561,4990550054026668580,2
1389183,1536239358,5399897164205922814,2
1429931,1536239269,190377194252611445,2


In [7]:
all_data[all_data.user_id == 50153023443529][['context_timestamp','item_brand_id' ,'user_' + 'category2_label' + '_future_1hour']]

,context_timestamp,item_brand_id,user_category2_label_future_1hour
190276,1536240332,5510998113992316938,3
265603,1536239858,8764113826665518348,4
475314,1536241480,7147251880558104750,0
543278,1536239177,7147251880558104750,7
683293,1536239046,8764113826665518348,8
832258,1536240799,6030170612722734811,1
1010442,1536239046,1326455302495217018,8
1369495,1536240561,4990550054026668580,2
1389183,1536239358,5399897164205922814,5
1429931,1536239269,190377194252611445,6


In [5]:
pd.set_option('display.max_rows', None)
all_data.dtypes

index                                                  int64
instance_id                                            int64
item_id                                                int64
item_brand_id                                          int64
item_city_id                                           int64
item_price_level                                       int64
item_sales_level                                       int64
item_collected_level                                   int64
item_pv_level                                          int64
user_id                                                int64
user_gender_id                                         int64
user_age_level                                         int64
user_occupation_id                                     int64
user_star_level                                        int64
context_id                                             int64
context_timestamp                                      int64
context_page_id         

In [10]:
all_data_path = feature_data_path + 'all_data_all_features.pkl'
all_data = load_pickle(all_data_path)
features = list(all_data.columns)

In [11]:
features

['index',
 'instance_id',
 'item_id',
 'item_brand_id',
 'item_city_id',
 'item_price_level',
 'item_sales_level',
 'item_collected_level',
 'item_pv_level',
 'user_id',
 'user_gender_id',
 'user_age_level',
 'user_occupation_id',
 'user_star_level',
 'context_id',
 'context_timestamp',
 'context_page_id',
 'shop_id',
 'shop_review_num_level',
 'shop_review_positive_rate',
 'shop_star_level',
 'shop_score_service',
 'shop_score_delivery',
 'shop_score_description',
 'is_trade',
 'day',
 'hour',
 'minute',
 'category2_label',
 'category3_label',
 'item_property_topic_k_10',
 'user_category2_label_click_day',
 'user_category3_label_click_day',
 'user_shop_id_click_day',
 'user_item_id_click_day',
 'user_item_brand_id_click_day',
 'user_context_page_id_click_day',
 'user_item_property_topic_k_10_click_day',
 'user_category2_label_click_hour',
 'user_category3_label_click_hour',
 'user_shop_id_click_hour',
 'user_item_id_click_hour',
 'user_item_brand_id_click_hour',
 'user_context_page_id

In [2]:
a = load_pickle(raw_data_path + 'all_data_6_7.pkl')
a

,index,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,...,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade,day,hour,minute,category2_label,category3_label
0,0,7118167540371,7595487976415452415,836752724084922533;8769426218101861255,7344985833148694227;6611726991947724280;783959...,144417789855323213,4228028106931716766,8,10,12,...,5009,0.974737,0.971962,0.972170,0,6,21,58,8769426218101861255,-1
1,1,43508800019098,5340996313837249587,836752724084922533;6670526099037031245,6241534230954727302;367082587220462692;2072967...,1975299596879116255,6219110439660858399,8,15,15,...,5015,0.963293,0.959580,0.979941,0,6,17,16,6670526099037031245,-1
2,2,78067858668749,8363638779510498577,836752724084922533;6693726201323251689,2072967855524022579;4621934203383159480;807039...,9167910794067700637,196257267849351217,8,10,13,...,5015,0.983985,0.983493,0.987826,0,6,20,51,6693726201323251689,-1
3,3,137525720365644,7121266959776152715,836752724084922533;6670526099037031245,6241534230954727302;367082587220462692;2072967...,9542632357298456,6219110439660858399,7,10,10,...,5014,0.964842,0.964313,0.978024,0,6,15,14,6670526099037031245,-1
4,4,151193932520734,5738516817201102842,836752724084922533;3613783563199627217;6370392...,2072967855524022579;2636395404473730413;772325...,7923860398679336202,6219110439660858399,7,11,14,...,5013,0.960387,0.958688,0.978505,0,6,16,53,3613783563199627217,6370392357088946840
5,5,191034067765499,3165167092384011466,836752724084922533;1367177154073382718,3657871859501171040;6491818071284064879;528103...,1078956017020681090,2225710480551643517,7,9,11,...,5011,0.977125,0.977785,0.975474,0,6,18,11,1367177154073382718,-1
6,6,221786632908010,2157590763338091652,836752724084922533;7314150500379498593,7126426653086863522;2636395404473730413;914848...,-1,3948283326616421003,5,14,13,...,5012,0.952626,0.955115,0.942377,0,6,20,58,7314150500379498593,-1
7,7,233375766178961,7296864533820220362,836752724084922533;5685690139879409547;7497531...,6241534230954727302;367082587220462692;5131280...,848910589365677287,8762827044490678569,8,9,11,...,5013,0.986273,0.987170,0.989783,0,6,19,25,5685690139879409547,7497531498747093265
8,8,235542268576836,3948722777398180782,836752724084922533;1909641874861640857,6241534230954727302;367082587220462692;5131280...,2564374947022447094,6219110439660858399,8,11,16,...,5015,0.969531,0.969538,0.982733,0,6,15,26,1909641874861640857,-1
9,9,271028956022181,4158769038163595899,836752724084922533;6693726201323251689,2072967855524022579;4067341101015777832;187732...,7756905285745693434,4899814843172066235,7,9,11,...,5014,0.963310,0.962803,0.974162,0,6,15,10,6693726201323251689,-1
